# Segmenting and Clustering Neighborhoods in Toronto

## 1. Build the code to scrape the following Wikipedia page

In [52]:
!pip install bs4
!pip install geocoder

In [53]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [54]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [55]:
soup = BeautifulSoup(html_data, 'html.parser')

## 2. Create the dataframe

In [155]:
data_pc = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood","Latitude","Longitude"])

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    
    if len(col)>0:
        pc =col[0].text.replace("\n", "")
        b =col[1].text.replace("\n", "")
        n =col[2].text.replace("\n", "")
    
        data_pc = data_pc.append({"PostalCode":pc, "Borough":b, "Neighborhood":n}, ignore_index=True)

data_pc = data_pc[data_pc.Borough != "Not assigned"]
data_pc = data_pc.groupby("PostalCode",as_index=False,sort=True).sum()
data_pc = data_pc.reset_index(drop=True)
# Chec if there is some not assigned neighborhoods
data_pc.loc[data_pc['Neighborhood'] == "Not assigned"]
# Show data shape
data_pc.shape

(103, 5)

## 3. Get longitude and latitude by useing Geocoder

In [57]:
import geocoder # import geocoder
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

In [60]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_fa9d07a7cae7449c9af7e35a6b5359de = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_fa9d07a7cae7449c9af7e35a6b5359de = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_fa9d07a7cae7449c9af7e35a6b5359de = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Mxq-DqyDnX4DL2AA_jSc2EqhiXtftDwnOOiWhqgPE1iv',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_fa9d07a7cae7449c9af7e35a6b5359de)

body = client_fa9d07a7cae7449c9af7e35a6b5359de.get_object(Bucket='datascience-donotdelete-pr-qg7cd0fgwpks0u',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
coordinates = pd.read_csv(body)

In [156]:
df = pd.DataFrame()
for index, row in data_pc.iterrows():
    postal = [row['PostalCode']]
    c_row  = coordinates[coordinates['Postal Code'].isin(postal)]
    latitude = c_row['Latitude'].values[0].astype(float)
    longitude = c_row['Longitude'].values[0].astype(float)
    row["Latitude"] = latitude
    row["Longitude"] = longitude
    
    df = df.append(row, ignore_index=True)

df.head(100)

,Borough,Latitude,Longitude,Neighborhood,PostalCode
0,Scarborough,43.806686,-79.194353,"Malvern, Rouge",M1B
1,Scarborough,43.784535,-79.160497,"Rouge Hill, Port Union, Highland Creek",M1C
2,Scarborough,43.763573,-79.188711,"Guildwood, Morningside, West Hill",M1E
3,Scarborough,43.770992,-79.216917,Woburn,M1G
4,Scarborough,43.773136,-79.239476,Cedarbrae,M1H
...,...,...,...,...,...
95,Etobicoke,43.643515,-79.577201,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",M9C
96,North York,43.756303,-79.565963,Humber Summit,M9L
97,North York,43.724766,-79.532242,"Humberlea, Emery",M9M
98,York,43.706876,-79.518188,Weston,M9N
